In [1]:
import os, sys 
%pwd

'd:\\collab_projects\\Crop_prediction\\notebooks'

In [2]:
os.chdir("../")
%pwd

'd:\\collab_projects\\Crop_prediction'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int 
    min_samples_split: int
    target_column: str

In [4]:
from cropPrediction.constants import * 
from cropPrediction.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            min_samples_split = params.min_samples_split,
            target_column = schema.name
            
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os
from cropPrediction import logger
from sklearn.ensemble import RandomForestClassifier
import joblib
import warnings
warnings.filterwarnings("ignore")

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        rfmodel = RandomForestClassifier(n_estimators=self.config.n_estimators, max_depth=self.config.max_depth, min_samples_split=self.config.min_samples_split, random_state=42)
        rfmodel.fit(train_x, train_y)

        joblib.dump(rfmodel, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-09-28 14:31:35,389: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-28 14:31:35,394: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-28 14:31:35,397: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-28 14:31:35,398: INFO: common: created directory at: artifacts]
[2023-09-28 14:31:35,400: INFO: common: created directory at: artifacts/model_trainer]
